In [ ]:
!unzip -uq "/content/all_image.zip" -d "/content/all_image"

In [ ]:
import keras
from keras import layers
from keras import models
import numpy as np
import tensorflow as tf

In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
background_path = '/content/all_image/background'
bloodvessel_path = '/content/all_image/bloodvessel'

background_images = [f for f in os.listdir(background_path) if os.path.isfile(os.path.join(background_path, f))]
bloodvessel_images = [f for f in os.listdir(bloodvessel_path) if os.path.isfile(os.path.join(bloodvessel_path, f))]

print(len(background_images))
print(background_images[3])
print(len(bloodvessel_images))
print(bloodvessel_images[3])

bloodvessel_image=[]
background_image=[]

for i in range(len(bloodvessel_images)):
    bloodvessel_image.append(cv2.imread(os.path.join(bloodvessel_path,bloodvessel_images[i]),cv2.IMREAD_COLOR))

for i in range(len(background_images)):
    background_image.append(cv2.imread(os.path.join(background_path,background_images[i]),cv2.IMREAD_COLOR))

bloodvessel_image=np.array(bloodvessel_image,dtype=np.float32)

background_image=np.array(background_image,dtype=np.float32)

print("damar boyut: ",bloodvessel_image.shape)
print("background boyutu",background_image.shape)

20000
background_image16718.bmp
20000
bloodvessel_image9214.bmp
damar boyut:  (20000, 32, 32, 3)
background boyutu (20000, 32, 32, 3)


In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(bloodvessel_image,background_image,train_size=0.8)

In [ ]:
print(y_train.shape)

(16000, 32, 32, 3)


In [ ]:
t=0
for i in range(len(x_train)):
  t+=1
  cv2.imwrite("/content/all_image/train/background/background_image"+str(t)+".bmp", y_train[i])
  cv2.imwrite("/content/all_image/train/bloodvessel/bloodvessel_image"+str(t)+".bmp", x_train[i])
  
for i in range(len(x_test)):
  t+=1
  cv2.imwrite("/content/all_image/validation/background/background_image"+str(t)+".bmp", y_test[i])
  cv2.imwrite("/content/all_image/validation/bloodvessel/bloodvessel_image"+str(t)+".bmp", x_test[i])


In [ ]:
!zip -r /content/fragmented_images.zip /content/all_image

In [ ]:
#indirilen dosyanın icerisinde sadece train ve validation klasörleri birakilir diger klasörler silinir.

from google.colab import files
files.download('/content/fragmented_images.zip') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!unzip -uq "/content/fragmented_image.zip" -d "/content/data"

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

main_dir = 'data/'
train_dir = os.path.join(main_dir,"train")
validation_dir = os.path.join(main_dir,"validation")

train_datagen = ImageDataGenerator(rescale=1./255, shear_range = 0.3,horizontal_flip=True, zoom_range = 0.4)

validation_datagen=ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(32,32), class_mode='categorical', batch_size=32, color_mode='rgb')

validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size=(32,32), class_mode='categorical', batch_size=32, color_mode='rgb')


print('x_train=%s, min=%.3f, max=%.3f' % (x_train.shape, x_train.min(), x_train.max()))

print('y_train=%s, min=%.3f, max=%.3f' % (y_train.shape, y_train.min(), y_train.max()))

Found 32000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
x_train=(16000, 32, 32, 3), min=0.000, max=255.000
y_train=(16000, 32, 32, 3), min=0.000, max=255.000


In [ ]:
x_train,y_train=x_train/255.0,y_train/255.0

x_train = np.expand_dims(x_train,axis=-1)
y_train = np.expand_dims(y_train,axis=-1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,Activation,BatchNormalization

model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(32,32,3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64,(5,5),activation='relu'))
model.add(MaxPooling2D((2, 2)))
#model.add(Conv2D(128,(5,5),activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
#model.add(Dense(64, activation='relu'))
model.add(Dense(2,activation='sigmoid'))

model.summary()

model.compile(loss='categorical_crossentropy',
  optimizer='adam',metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               160100    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2

In [ ]:
# eğitim
batch_size = 32

hist = model.fit(
        train_generator,
        steps_per_epoch = 500,
        epochs=100,
        validation_data = validation_generator,
        validation_steps = 20)

Epoch 1/100
500/500 [==============================] - 39s 77ms/step - loss: 0.6591 - accuracy: 0.5845 - val_loss: 0.6069 - val_accuracy: 0.6359
Epoch 2/100
500/500 [==============================] - 41s 83ms/step - loss: 0.6378 - accuracy: 0.5953 - val_loss: 0.6166 - val_accuracy: 0.6094
Epoch 3/100
500/500 [==============================] - 38s 75ms/step - loss: 0.6324 - accuracy: 0.6057 - val_loss: 0.6537 - val_accuracy: 0.5719
Epoch 4/100
500/500 [==============================] - 39s 79ms/step - loss: 0.6353 - accuracy: 0.5917 - val_loss: 0.6254 - val_accuracy: 0.6141
Epoch 5/100
500/500 [==============================] - 39s 79ms/step - loss: 0.6308 - accuracy: 0.6045 - val_loss: 0.6288 - val_accuracy: 0.5828
Epoch 6/100
500/500 [==============================] - 37s 75ms/step - loss: 0.6282 - accuracy: 0.6042 - val_loss: 0.6087 - val_accuracy: 0.6203
Epoch 7/100
500/500 [==============================] - 37s 75ms/step - loss: 0.6342 - accuracy: 0.5959 - val_loss: 0.6400 - val_ac

KeyboardInterrupt: ignored